### Key Features (Independent Variables):

Traffic load, slice type, bandwidth demand, network latency, jitter, CPU/memory usage.

### Target Output (Dependent Variable):

Optimal resource allocation (bandwidth, CPU, memory).

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("network_slicing_dataset.csv")

In [3]:
df.head(2)

,Traffic_Volume,Packet_Arrival_Rate,Latency_Requirement,Jitter_Requirement,Packet_Loss_Tolerance,CPU_Utilization,Memory_Utilization,Bandwidth_Utilization,Node_Energy_Level,Num_Active_Users,...,Interference_Level,Slice_Type,Allocated_Bandwidth,Allocated_CPU,Allocated_Memory,Achieved_Latency,Achieved_Jitter,Achieved_Packet_Loss,Slice_Throughput,Slice_Energy_Consumption
0,380.794718,193.084001,73.269833,6.417631,1.494560,77.778926,69.324416,13.412888,83.634801,35,...,16.467206,URLLC,206.402199,52.332272,59.084929,80.045690,6.368104,1.320867,344.736042,32.877502
1,951.207163,173.126927,19.266688,4.646995,0.474089,49.561363,80.488150,76.280370,23.074279,33,...,2.433201,eMBB,580.473710,42.878339,41.194318,18.375518,4.952067,0.484651,881.196505,6.707558


In [4]:
df.columns

Index(['Traffic_Volume', 'Packet_Arrival_Rate', 'Latency_Requirement',
       'Jitter_Requirement', 'Packet_Loss_Tolerance', 'CPU_Utilization',
       'Memory_Utilization', 'Bandwidth_Utilization', 'Node_Energy_Level',
       'Num_Active_Users', 'Link_Quality', 'Interference_Level', 'Slice_Type',
       'Allocated_Bandwidth', 'Allocated_CPU', 'Allocated_Memory',
       'Achieved_Latency', 'Achieved_Jitter', 'Achieved_Packet_Loss',
       'Slice_Throughput', 'Slice_Energy_Consumption'],
      dtype='object')

In [5]:
input_features = [
    'Traffic_Volume',
    'Packet_Arrival_Rate',
    'Latency_Requirement',
    'Jitter_Requirement',
    'Packet_Loss_Tolerance',
    'CPU_Utilization',
    'Memory_Utilization',
    'Bandwidth_Utilization',
    'Num_Active_Users',
    'Slice_Type'
]
output_features = [
    'Allocated_Bandwidth',
    'Allocated_CPU',
    'Allocated_Memory'
]

In [6]:
required_columns=[
    'Traffic_Volume',
    'Packet_Arrival_Rate',
    'Latency_Requirement',
    'Jitter_Requirement',
    'Packet_Loss_Tolerance',
    'CPU_Utilization',
    'Memory_Utilization',
    'Bandwidth_Utilization',
    'Num_Active_Users',
    'Slice_Type',
    'Allocated_Bandwidth',
    'Allocated_CPU',
    'Allocated_Memory'
]

In [7]:
df=df[required_columns]

In [8]:
df.shape

(10000, 13)

In [9]:
df.isnull().sum()

,0
Traffic_Volume,0
Packet_Arrival_Rate,0
Latency_Requirement,0
Jitter_Requirement,0
Packet_Loss_Tolerance,0
CPU_Utilization,0
Memory_Utilization,0
Bandwidth_Utilization,0
Num_Active_Users,0
Slice_Type,0


In [10]:
x=df.drop(output_features,axis=1)
y=df.drop(input_features,axis=1)

In [11]:
print(x.shape,y.shape)

(10000, 10) (10000, 3)


In [12]:
x.head(2)

,Traffic_Volume,Packet_Arrival_Rate,Latency_Requirement,Jitter_Requirement,Packet_Loss_Tolerance,CPU_Utilization,Memory_Utilization,Bandwidth_Utilization,Num_Active_Users,Slice_Type
0,380.794718,193.084001,73.269833,6.417631,1.494560,77.778926,69.324416,13.412888,35,URLLC
1,951.207163,173.126927,19.266688,4.646995,0.474089,49.561363,80.488150,76.280370,33,eMBB


In [13]:
y.head(2)

,Allocated_Bandwidth,Allocated_CPU,Allocated_Memory
0,206.402199,52.332272,59.084929
1,580.473710,42.878339,41.194318


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [16]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [18]:
preprocessor=ColumnTransformer(transformers=[
    ('scaler',StandardScaler(),['Traffic_Volume','Packet_Arrival_Rate','Latency_Requirement','Jitter_Requirement','Packet_Loss_Tolerance','CPU_Utilization','Memory_Utilization','Bandwidth_Utilization','Num_Active_Users']),
    ('encoder',OneHotEncoder(),['Slice_Type'])
])

In [19]:
x_trian_processed=preprocessor.fit_transform(x_train)

In [20]:
x_test_processed=preprocessor.transform(x_test)

In [21]:
x_trian_processed.shape

(7000, 12)

In [22]:
x_trian_processed[0]

array([-1.25208642, -1.33426642,  1.19597147, -1.50722075, -0.23059535,
       -0.59207916,  0.88456794,  1.38546202,  0.08524298,  0.        ,
        1.        ,  0.        ])

In [23]:
from keras.models import Sequential
from keras.layers import Dense

In [24]:

model = Sequential([
    Dense(128, activation='relu', input_shape=(x_trian_processed.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='linear')  # 3 outputs for bandwidth, CPU, memory
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [26]:
model.fit(x_trian_processed, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 46146.4570 - mae: 121.0918 - val_loss: 3212.1328 - val_mae: 37.3851
Epoch 2/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2718.2056 - mae: 35.6123 - val_loss: 2126.9875 - val_mae: 31.5508
Epoch 3/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2145.3840 - mae: 31.0540 - val_loss: 1841.6937 - val_mae: 26.4231
Epoch 4/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1828.3962 - mae: 25.8270 - val_loss: 1701.5752 - val_mae: 23.7787
Epoch 5/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1736.5304 - mae: 23.7017 - val_loss: 1609.7501 - val_mae: 22.4331
Epoch 6/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1672.2275 - mae: 22.4972 - val_loss: 1572.7356 - val_mae: 21.7444
Epoch 7/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1600.1761 - mae: 21.7976 - val_loss: 1521.1027 - val_mae: 21.2389
Epoch 8/50
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1555.2655 - mae: 21.2092 - val_loss: 1528.9283 - val_mae

In [27]:
loss, mae = model.evaluate(x_test_processed, y_test)
print(f'Test Loss: {loss:.4f}, Test MAE: {mae:.4f}')


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1572.6365 - mae: 21.0219
Test Loss: 1562.1306, Test MAE: 20.8498


In [29]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [31]:
with open("preprocessor.pkl","wb") as f :
  pickle.dump(preprocessor,f)

In [32]:
with open("model.pkl","wb") as f :
  pickle.load(model.f)

with open("preprocessor.pkl","wb") as f :
  pickle.load(preprocessor,f)



AttributeError: 'Sequential' object has no attribute 'f'

In [ ]:
df

In [33]:
processed_data=preprocessor.transform(df)

In [34]:
model.predict(processed_data)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([[264.4509  ,  54.211006,  45.52767 ],
       [664.69574 ,  36.11185 ,  55.517967],
       [508.91437 ,  18.658628,  32.559776],
       ...,
       [665.16534 ,  21.96039 ,  56.941216],
       [290.83368 ,  23.962635,  13.048349],
       [143.77208 ,  36.877495,   9.697168]], dtype=float32)